In [2]:
import os.path
import numpy as np
import copy

arquivo = os.path.join('projeto.csv') 

dataRDD = (sc.textFile(arquivo, 8))

In [3]:
lpRDD = dataRDD.map(lambda l: l.split(';')).map(lambda l: {'label':float(l[0]),'features':[float(l[1]),float(l[2]),l[3]]})

In [4]:
def contador(lp, feats, splits):
    r = []
    for feat in feats:
        splitss = splits[feat]
        for split in splitss:
            if feat==3:
                r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
            elif feat==0 or feat==1:
                if lp['features'][feat] >= split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
            elif feat==2:                
                if lp['features'][feat] in split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
    return r

In [5]:
def soma(t1,t2):
    return  ((t1[0][0]+t2[0][0],t1[0][1]+t2[0][1]), #((0.668333333, 0), (0.44666944399888886, 0), (1, 0))
             (t1[1][0]+t2[1][0],t1[1][1]+t2[1][1]),
             (t1[2][0]+t2[2][0],t1[2][1]+t2[2][1]))

In [6]:
def calc_stats(t):
    if t[1][2][0] > 0:
        mmenor = t[1][0][0] / t[1][2][0]
        sdmenor = (t[1][1][0] - (t[1][0][0]**2 / t[1][2][0]))/(t[1][2][0])
        cvmenor = np.sqrt(sdmenor)/mmenor*100
        menor = (sdmenor, cvmenor,t[1][2][0])
    else:
        menor = (0, 0, 0)
    
    if t[1][2][1] > 0:    
        mmaior = t[1][0][1] / t[1][2][1]
        sdmaior = (t[1][1][1] - (t[1][0][1]**2 / t[1][2][1]))/(t[1][2][1])
        cvmaior = np.sqrt(sdmaior)/mmaior*100
        maior = (sdmaior, cvmaior,t[1][2][1])
    else:
        maior = (0, 0, 0)
    return (t[0], menor, maior)

In [8]:
def sdreduction(t,c):
    #return ((t[0], c[2][0] - t[1][0]*(t[1][2]/c[2][2]) - t[2][0]*(t[2][2]/c[2][2]))) 
    return ((t[0], c[2][0] - (t[1][2]/c[2][2])*t[1][0] - (t[2][2]/c[2][2])*t[2][0], (t[1][1], t[2][1]))) 

In [9]:
def prepara(t):
    
    return ((t['features'][0],t['features'][0]),(t['features'][1],t['features'][1]), set((t['features'][2],)))
    
def minmax(t1,t2):
    return ((min(t1[0][0],t2[0][0]),max(t1[0][1],t2[0][1])),
            (min(t1[1][0],t2[1][0]),max(t1[1][1],t2[1][1])),
             t1[2].union(t2[2]))

In [13]:
import time
def tree(lpRDD, maxDepth=1, currDepth=0, caminho=[], st=None,direction='root',verbose=1):
    if st!=None:
        if st[0]<10 or maxDepth<currDepth or st[1]<0:
            media = lpRDD.map(lambda x: x['label']).mean()
            if verbose >= 2:
                print('retorno: caminho: %s média: %s'%(caminho,media))
            return [caminho, (direction, media)]    
    
    t0 = time.time()
    if verbose >= 2:
        print ('total: %s'%lpRDD.count())
    #min0 = lpRDD.map(lambda l: l['features'][0]).min()
    #print ('calc min att 0: %s s'%(time.time()-t0))
    #max0 = lpRDD.map(lambda l: l['features'][0]).max()
    #print ('calc max att 0: %s s'% (time.time()-t0))
    #min1 = lpRDD.map(lambda l: l['features'][1]).min()
    #print ('calc min att 1: %s s'% (time.time()-t0))
    #max1 = lpRDD.map(lambda l: l['features'][1]).max()
    (min0,max0),(min1,max1),periods =lpRDD.map(prepara).reduce(minmax)
    if verbose >= 1:
        print ( "min-max-uniques:", (min0,max0),(min1,max1),periods)
    if verbose >= 2:
        print ('calc minmax: %.2f s'%(time.time()-t0))
    #periods = lpRDD.map(lambda l: l['features'][2]).distinct().collect()
    periods = list(periods)

    #lines = lpRDD.map(lambda l: l['features'][3]).distinct().collect()
    distsplits = [i*(max0-min0)/5. for i in range(1,5)]
    timesplits = [i*(max1-min1)/5. for i in range(1,5)]    
    perisplits = [tuple([periods[j] for j in range(i+1)]) for i in range(len(periods)-1)]

    feats = [0,1,2,3]
    splits = [distsplits,timesplits,perisplits,['full']] 
    results = lpRDD.flatMap(lambda lp: contador(lp,feats,splits)).reduceByKey(soma)
    if verbose >= 2:
        print ('contador: %.2f s'%(time.time()-t0))
    statistics = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]<3)
    if verbose >= 2:
        print ('estatísticas: %.2f s'%(time.time()-t0))
    statistics3 = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]==3)
    classe = statistics3.collect()[0]
    es = statistics.map(lambda x: sdreduction(x,classe)).collect()  
    e = max(es, key = lambda t: t[1]) #atributo / split escolhido
    if verbose >= 2:
        print ('atributo escolhido: %.2f s'%(time.time()-t0))
    if e[0][0]<2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]<e[0][1])
            if verbose >= 2:
                print ('filtro menor01: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]>e[0][1])   
            if verbose >= 2:
                print ('filtro maior01: %.2f s'%(time.time()-t0))
    elif e[0][0]==2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] in e[0][1])
            if verbose >= 2:
                print ('filtro menor2: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] not in e[0][1])
            if verbose >= 2:
                print ('filtro maior2: %.2f s'%(time.time()-t0))
    
    caminho.append((direction,e[0]))
    camenor = copy.deepcopy(caminho)
    camaior = copy.deepcopy(caminho)
    

    amenor = tree(menorRDD, maxDepth, currDepth+1,camenor, (e[2][0], e[1]),'menor',verbose=2)
  

    amaior = tree(maiorRDD, maxDepth, currDepth+1,camaior, (e[2][1], e[1]),'maior',verbose=2)
    return [amenor, amaior]

In [14]:
ti=time.time()
arvore = tree(lpRDD,maxDepth=4,verbose=2)
tf = time.time()

total: 1048575
min-max-uniques: (0.0003, 17.64153) (0.0, 13.29138889) {'n', 'ep', 'm', 'pt', 'pm'}
calc minmax: 7.32 s
contador: 7.36 s
estatísticas: 7.36 s
atributo escolhido: 28.67 s
filtro menor01: 28.67 s
filtro maior01: 28.67 s
total: 1024936
min-max-uniques: (0.0003, 17.64153) (0.0, 2.658055556) {'n', 'ep', 'm', 'pt', 'pm'}
calc minmax: 7.39 s
contador: 7.42 s
estatísticas: 7.42 s
atributo escolhido: 30.50 s
filtro menor01: 30.50 s
filtro maior01: 30.50 s
total: 1011609
min-max-uniques: (0.0003, 17.64153) (0.0, 1.594722222) {'n', 'ep', 'm', 'pt', 'pm'}
calc minmax: 7.79 s
contador: 7.82 s
estatísticas: 7.82 s
atributo escolhido: 29.99 s
filtro menor01: 29.99 s
filtro maior01: 29.99 s
total: 994134
min-max-uniques: (0.0003, 17.64153) (0.0, 1.275555556) {'n', 'ep', 'm', 'pt', 'pm'}
calc minmax: 8.45 s
contador: 8.47 s
estatísticas: 8.47 s
atributo escolhido: 29.57 s
filtro menor01: 29.57 s
filtro maior01: 29.57 s
total: 942267
min-max-uniques: (0.0003, 17.64069) (0.0, 1.020277778) 

retorno: caminho: [('root', (1, 2.658277778)), ('maior', (2, ('n',))), ('menor', (1, 4.7731111111999995)), ('maior', (0, 13.040608)), ('menor', (0, 5.048088))] média: 6.806723856200535
total: 5016
min-max-uniques: (13.0741, 16.68459) (4.773611111, 8.625277778) {'n'}
calc minmax: 5.40 s
contador: 5.42 s
estatísticas: 5.42 s
atributo escolhido: 8.90 s
filtro menor01: 8.90 s
filtro maior01: 8.90 s
retorno: caminho: [('root', (1, 2.658277778)), ('maior', (2, ('n',))), ('menor', (1, 4.7731111111999995)), ('maior', (0, 13.040608)), ('maior', (0, 1.4441960000000003))] média: 0.0
retorno: caminho: [('root', (1, 2.658277778)), ('maior', (2, ('n',))), ('menor', (1, 4.7731111111999995)), ('maior', (0, 13.040608)), ('maior', (0, 1.4441960000000003))] média: 7.395261884250793
total: 12394
min-max-uniques: (0.13708, 17.63388) (2.658333333, 13.29138889) {'ep', 'm', 'pt'}
calc minmax: 5.89 s
contador: 5.91 s
estatísticas: 5.91 s
atributo escolhido: 9.43 s
filtro menor01: 9.43 s
filtro maior01: 9.43 s


In [15]:
print('Tempo total de construção: %.2f'%(tf-ti))

Tempo total de construção: 490.85


In [16]:
arvore

[[[[[[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('menor', (2, ('n', 'ep', 'm')))],
      ('menor', 1.0017964003506334)],
     [[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('menor', (2, ('n', 'ep', 'm')))],
      ('maior', 1.1339875351100666)]],
    [[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('maior', (2, ('n',)))],
      ('menor', 3.398222784192883)],
     [[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1, 1.2757777775999999)),
       ('menor', (1, 1.0204444448)),
       ('maior', (2, ('n',)))],
      ('maior', 1.465807749419039)]]],
   [[[[('root', (1, 2.658277778)),
       ('menor', (1, 1.5948333336)),
       ('menor', (1